In [20]:
!pip install pymysql

In [ ]:
import pymysql
import pandas as pd
import requests, json

In [ ]:
toilet = pd.read_excel('./광주_화장실.xlsx')

toilet.drop(['번호', '구분', '화장실명', '소재지지번주소', '남성용-대변기수', '남성용-소변기수', '남성용-장애인용대변기수', '남성용-장애인용소변기수',
             '남성용-어린이용대변기수', '남성용-어린이용소변기수', '여성용-대변기수', '여성용-장애인용대변기수', '여성용-어린이용대변기수',
             '관리기관명', '전화번호', '개방시간', '설치연월', 'WGS84위도', 'WGS84경도', '화장실소유구분', '화장실설치장소유형', '오물처리방식',
             '비상벨설치여부', '비상벨설치장소', '화장실입구CCTV설치유무', '기저귀교환대유무', '기저귀교환대장소', '리모델링연월', '데이터기준일자'], axis=1, inplace=True)

toilet.dropna(subset=['소재지도로명주소'], axis=0, inplace=True)

print(len(toilet))

mydb = pymysql.connect(
    host='database-plonit.clluyrc4oifq.ap-northeast-2.rds.amazonaws.com',
    user='plonit',
    password='DB 비밀번호',
    port=3306,
    db='plonit'
)

dbconn = mydb.cursor()

sql = 'INSERT INTO item_info (gugun_code, content_type, latitude, longitude) values (%s, false, %s, %s)'

for idx, row in toilet.iterrows():
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + row[0]
        headers = {"Authorization": "KakaoAK 카카오개인키"}
        api_json = json.loads(str(requests.get(url,headers=headers).text))

        if len(api_json['documents']) == 0:
            continue

        address = api_json['documents'][0]['address']
        h_code = address['h_code'][:5] + ('0' * 5)
        lat = float(address['y'])
        lng = float(address['x'])

        dbconn.execute(sql, (h_code, lat, lng))

    except:
        print(row[0])

mydb.commit()
mydb.close()

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1344
광주광역시 서구 하남대로 710번길


In [ ]:
()